In [1]:
import json
import matplotlib.pyplot as plt
import statistics

with open('new_train.json', 'rt') as f:
  queries = json.load(f)

In [2]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)          
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [3]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://34.135.82.254:8080'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_anchor', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))

In [ ]:
qs_res

In [4]:
queries, durations,average_precisions = zip(*qs_res)

In [ ]:
average_duration = statistics.mean(durations)
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(queries, durations, 'r-', label="Duration")
ax.scatter(queries, durations, color='r', label="Duration")
ax.axhline(average_duration, color='r', linestyle='--', label="Average Duration")
ax.set_xlabel('Query')
ax.set_ylabel('Duration (s)')
ax.set_xticklabels(queries, rotation=90)
plt.title("Title Weight=1 Stemming=True\nAverage Duration="+str(average_duration))
#plt.savefig("t1_stem_dur.png")
plt.show()

In [ ]:
queries_n = [query for query,ap in zip(queries, average_precisions) if ap is not None]
average_precisions_n = [ap for ap in average_precisions if ap is not None]
average_precision = statistics.mean(average_precisions_n)
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(queries_n, average_precisions_n, 'b-', label="Average Precision")
ax.scatter(queries_n, average_precisions_n, color='b', label="Average Precision")
ax.axhline(average_precision, color='b', linestyle='--', label="Average Precision")
ax.set_xlabel('Query')
ax.set_ylabel('Average Precision')
ax.set_xticklabels(queries, rotation=90)
plt.title("Title Weight=1, Stemming_Title=True\nAverage Precision="+str(average_precision))
#plt.savefig("t1_stem_ap.png")
plt.show()

In [17]:
import requests
pv = requests.post('http://34.135.82.254:8080/get_pageview', json=[57069491,65967176,42163310])
pr = requests.post('http://34.135.82.254:8080/get_pagerank', json=[57069491,65967176,42163310])

In [20]:
pr.content

b'[\n  0.33356282489865574, \n  0.1548102087544133, \n  3.102288653188671\n]\n'

In [21]:
pv.content

b'[\n  21314, \n  5962, \n  1044471\n]\n'